In [8]:
# !pip download sentence-transformers -d ./sentence_transformers/
! ls ../input/sentence-transformers/sentence_transformers

MarkupSafe-3.0.2-cp39-cp39-win_amd64.whl
PyYAML-6.0.2-cp39-cp39-win_amd64.whl
certifi-2024.8.30-py3-none-any.whl
charset_normalizer-3.4.0-cp39-cp39-win_amd64.whl
colorama-0.4.6-py2.py3-none-any.whl
filelock-3.16.1-py3-none-any.whl
fsspec-2024.10.0-py3-none-any.whl
huggingface_hub-0.26.3-py3-none-any.whl
idna-3.10-py3-none-any.whl
jinja2-3.1.4-py3-none-any.whl
joblib-1.4.2-py3-none-any.whl
mpmath-1.3.0-py3-none-any.whl
networkx-3.2.1-py3-none-any.whl
numpy-2.0.2-cp39-cp39-win_amd64.whl
packaging-24.2-py3-none-any.whl
pillow-11.0.0-cp39-cp39-win_amd64.whl
regex-2024.11.6-cp39-cp39-win_amd64.whl
requests-2.32.3-py3-none-any.whl
safetensors-0.4.5-cp39-none-win_amd64.whl
scikit_learn-1.5.2-cp39-cp39-win_amd64.whl
scipy-1.13.1-cp39-cp39-win_amd64.whl
sentence_transformers-3.3.1-py3-none-any.whl
sympy-1.13.1-py3-none-any.whl
threadpoolctl-3.5.0-py3-none-any.whl
tokenizers-0.20.3-cp39-none-win_amd64.whl
torch-2.5.1-cp39-cp39-win_amd64.whl
tqdm-4.67.1-py3-none-any.whl
transformers-4.46.3-py3-no

In [13]:
!pip install sentence-transformers --no-index --find-links=file:/kaggle/input/sentence-transformers/sentence_transformers 

Looking in links: file:///kaggle/input/sentence-transformers/sentence_transformers
Processing /kaggle/input/sentence-transformers/sentence_transformers/sentence_transformers-3.3.1-py3-none-any.whl


In [14]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import kagglehub
import csv
import os

import numpy as np
import pandas as pd
import torch
from torch.nn import functional
from tqdm import tqdm

from transformers import AutoTokenizer, T5Tokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [15]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

train_df = pd.read_csv('/kaggle/input/llm-prompt-recovery/train.csv')
test_df = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')

train_df.head()

,id,original_text,rewrite_prompt,rewritten_text
0,-1,The competition dataset comprises text passage...,"Convert this into a sea shanty: """"""The competi...",Here is your shanty: (Verse 1) The text is rew...


In [16]:
EPOCHS = 1
BATCH_SIZE = 32
LEARNING_RATE = 1e-5

max_new_tokens = 30
max_sentences_in_response = 1

In [17]:
# Prompt_dataset
path = kagglehub.dataset_download("what5up/concat-prompts")
files = os.listdir(path)
csv_file = [file for file in files if file.endswith('.csv')][0]
data = pd.read_csv(os.path.join(path, csv_file))
prompt_dataset = data['prompt'].tolist()

In [18]:
# Vocabulary dataset
tokenizer = T5Tokenizer.from_pretrained("sentence-transformers/sentence-t5-base")
vocabulary_dataset = tokenizer.get_vocab()
vocabulary_dataset.pop('<pad>', None)
vocabulary_dataset.pop('</s>', None)
vocabulary_dataset.pop('<unk>', None)
vocabulary_dataset = list(vocabulary_dataset.keys())

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [19]:
print(prompt_dataset[:5])
print(vocabulary_dataset[:5])

['Convert the text into a vintage circus poster announcement', "Convert the text into a social media platform's community guidelines", 'Rewrite this as a college course description.', 'Rephrase this as a debate on furniture rights, featuring chairs.', "Make the text into a home improvement expert's tips for a bathroom remodel"]
['▁', 'X', '.', ',', 's']


In [31]:
def hotflip(prompt_tokens, token_index, vocabulary_dataset, model, target_embeddings):
    possible_prompts = [' '.join(prompt_tokens[:token_index] + [vocab] + prompt_tokens[token_index + 1:]) for vocab in vocabulary_dataset]
    prompt_embeddings = torch.tensor(np.array([model.encode(prompt) for prompt in possible_prompts]), dtype=torch.float32)

    simularities = cosine_similarity(target_embeddings, prompt_embeddings)

    vocab_index = simularities.argmax().item()
    prompt_tokens[token_index] = vocabulary_dataset[vocab_index]

    return prompt_tokens

In [20]:
# Model
model_name = "sentence-transformers/sentence-t5-base"
model = SentenceTransformer(model_name)
for param in model.parameters():
    param.requires_grad = False

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

rust_model.ot:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [21]:
print(model)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
)


In [23]:
# Initial prompt
prompt = "Rewrite this text to make it more helpful"
prompt_embeddings = model.encode(prompt)
prompt_embeddings = torch.tensor(prompt_embeddings, dtype=torch.float32).unsqueeze(0)
print("Initial prompt")
# Prompt tokens
prompt_tokens = prompt.split(' ')
print("Prompt tokens")
# Target
target_embeddings = model.encode(prompt_dataset)
target_embeddings = torch.tensor(target_embeddings, dtype=torch.float32).mean(dim=0).unsqueeze(0)

print(prompt_embeddings.shape)
print(target_embeddings.shape)

In [34]:
# meam prompt
fixed_output = ""

In [ ]:
for epoch in range(EPOCHS):
    for token_index in range(len(prompt_tokens)):
        loss =  1 - cosine_similarity(target_embeddings, prompt_embeddings)
        prompt_tokens = hotflip(prompt_tokens, token_index, vocabulary_dataset, model, target_embeddings)
        prompt = ' '.join(prompt_tokens)
        prompt_embeddings = model.encode(prompt)
        prompt_embeddings = torch.tensor(prompt_embeddings, dtype=torch.float32).unsqueeze(0)

    print(f"Epoch: {epoch}, Loss: {loss.item()}")
    print(prompt)
    fixed_output = prompt

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# 將所有樣本的 rewrite_prompt 設定為 meam prompt
test_df['rewrite_prompt'] = [fixed_output] * len(train_df)

test_df[['id', 'rewrite_prompt']].head()


In [ ]:
test_df[['id', 'rewrite_prompt']].to_csv('submission.csv', index=False)
print("Submission file created successfully!")